# object selection form catalog

currently it's just a simple selection S/N of Ha Hb fluxes > 5

In [ ]:
from   astropy.table import Table, vstack, join, setdiff
import numpy         as np
from   astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u
import warnings
from   astropy.io.fits.card import VerifyWarning
warnings.simplefilter('ignore', VerifyWarning)

### Goodnorth sample prearation, agn selection

In [90]:
# initial catalog
gn   = Table.read('hlsp_clear_hst_wfc3_gdn_multi_v4.1_clear.fits')
gn_m = Table.read('hlsp_clear_hst_wfc3-acs_gdn-3dhst_multi_v4.6_zout.fits')
gn_cat = join(gn,gn_m,keys_left='ID',keys_right='id',join_type='left',metadata_conflicts='silent')
#agn catalog crossmatch
filename = "goodsn_agn.txt"
with open(filename, "r") as file:
    source_ids = file.readlines()
source_ids = [line.strip() for line in source_ids]
coord_agn = SkyCoord(source_ids,unit=(u.hourangle, u.deg))
coord_obj = SkyCoord(gn_cat['ra'],gn_cat['dec'],unit=(u.deg,u.deg))
idx,d2d,d3d = match_coordinates_sky(coord_agn,coord_obj)
gn_cat.remove_rows()
['agn' for index in idx[d2d.deg < 5/3600] else '/' for index in range(len(gn_cat))]

SyntaxError: invalid syntax (3195414558.py, line 14)

In [ ]:

gs   = Table.read('hlsp_clear_hst_wfc3_gds_multi_v4.1_clear.fits')
gs_m = Table.read('hlsp_clear_hst_wfc3-acs_gds-3dhst_multi_v4.6_zout.fits')
gs_agn = Table.read('goodss_agn.txt', 
        format='ascii',  
        names=[
            "3DHST", "VLA", "CDF-S", "z", "RAdeg", "DEdeg", "f_z", "mtype-s", "mtype-c",
            "xtype-l", "xtype-x", "rtype-r", "rtype-f", "otype-sp", "otype-se", "var",
            "logL-i", "f_logL-i", "logL-s", "logL-x", "Mbh", "M*", "rTag", "xTag", "mTag",
            "oTag", "logNH", "logLx", "logLint", "gamma", "er_gamma", "CType", "L3GHz",
            "L6GHz", "q_L3GHz", "q_L6GHz", "q24Obs", "e_q24Obs", "q24crt", "alpha",
            "er_alpha", "tau", "logL5100", "logL3um", "logL6um", "logL8um", "logL12um",
            "logL20um"
        ]
)
gs_cat = join(gs,gs_m,keys_left='ID',keys_right='id',join_type='left',metadata_conflicts='silent')
gs_cat['tag'] = ['agn' if id in gs_agn['3DHST'] else '/' for id in gs['ID']]



cat_lis = vstack([gn_cat,gs_cat],metadata_conflicts='silent')
cat_lis.write('full_object_catalog.fits',overwrite=True)
cat_lis = gn_cat
cat_hasline = cat_lis[np.logical_and(cat_lis['Ha_FLUX'] >0, cat_lis['Hb_FLUX']>0)]
sn_ha = cat_hasline['Ha_FLUX']/cat_hasline['Ha_FLUX_ERR']
sn_hb = cat_hasline['Hb_FLUX']/cat_hasline['Hb_FLUX_ERR']
selection = np.logical_and(sn_ha>5,sn_hb>5)
obj_lis = cat_hasline[selection]

#obj_lis.write('obj_lis_selected.fits',overwrite=True)

print('total num of objs in the fields',len(cat_lis))
print('num of objs after s/n selection:',len(obj_lis))

import matplotlib.pyplot as plt
plt.plot(obj_lis['ra'],obj_lis['dec'],linewidth=0,marker='.')

TypeError: Row index must be an integer

In [ ]:
d2d.deg < 6/3600

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False])

# download spectrum from server.

It is recommended that to run the script downloadSpectra in prompt because of potential stability issue

# Extract line maps from data products

In [2]:
from    astropy.table       import Table
import  numpy               as     np
from    astropy.io          import fits
from    photutils.psf       import matching as match
from    astropy.convolution import convolve_fft 
from    tqdm                import tqdm
import  os
import  gc                                         

#this gives the file name + prefix of an obj in the cat file
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"


def extract_HaHb(hdu):
    """

    pass objs from obj_lis to extract ha hb lines

    return: HDUlist with the following entry:

    0 primary extension, same as original file

    1 line-fit results

    2 segmentation map

    3 clear filter maps

    4,5 Ha line map & line weight

    6,7 Hb line map & line weight

    """
    #set up a crop of 50x50 pix in the center
    center_size = 50; shape = hdu[5].shape[0]
    #start index: si and end index: ei
    si = (shape - center_size) // 2; 
    ei = si + center_size

    new_file = fits.HDUList()
    #save primary extension
    new_file.append(hdu[0])
    #save line-fit info
    new_file.append(hdu[1])
    """
    select segmentation map [4]
    also save 1 DSCI image for comparison [5]
    """
    for i in [4,5]: 
        hdu[i].data = hdu[i].data[si:ei,si:ei]
        new_file.append(hdu[i])

    #loop to select ha hb line maps
    for image in hdu:
        if image.header.get('EXTTYPE') in ['Ha','Hb'] and (image.name == 'LINE' or image.name == 'LINEWHT'):
            image.data = image.data[si:ei,si:ei]
            image.name = f"{image.name}_{image.header['EXTTYPE']}"
            new_file.append(image)
    return new_file


def data_process(obj):
    try:
        path_data_product   = f"data_products/{file_name(obj,'full')}"
        path_data_extracted = f"data_extracted/{file_name(obj,'extracted')}"

        need_update_from_data_products = True
        need_save_file = True

        with fits.open(path_data_product) as hdu:
            extracted = extract_HaHb(hdu)

        #this part still needs psf matching---------
        
        #save file
            extracted.writeto(path_data_extracted,overwrite=True)
        return f"{obj['subfield']}-{obj['ID']} saved"

    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error:{e}"


from concurrent.futures import ThreadPoolExecutor, as_completed
def main():
    os.makedirs('data_extracted',exist_ok=True)
    obj_lis = Table.read('obj_lis_selected.fits')
    results = []
    max_threads= 7
    if max_threads > 1:
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(data_process,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
    else:
        for obj in tqdm(obj_lis):
            results.append(data_process(obj))
            
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

if __name__ == '__main__':
    main()


Processing: 100%|██████████| 158/158 [01:36<00:00,  1.64it/s]

['GN2-11228 saved(updated)', 'GN7-13686 saved(updated)', 'GN7-12769 saved(updated)', 'GN7-11883 saved(updated)', 'GN7-13197 saved(updated)', 'GN2-10512 saved(updated)', 'GN7-11839 saved(updated)', 'GN7-14716 saved(updated)', 'GN7-13777 saved(updated)', 'GN7-13909 saved(updated)', 'GN7-14184 saved(updated)', 'GN2-14895 saved(updated)', 'GN7-14281 saved(updated)', 'GN7-14850 saved(updated)', 'GN7-15204 saved(updated)', 'GN7-15127 saved(updated)', 'GN7-16041 saved(updated)', 'GN7-15300 saved(updated)', 'GN2-16173 saved(updated)', 'GN2-16752 saved(updated)', 'GN7-15761 saved(updated)', 'GN2-17829 saved(updated)', 'GN2-17579 saved(updated)', 'GN4-19075 saved(updated)', 'GN2-18224 saved(updated)', 'GN7-19005 saved(updated)', 'GN7-19235 saved(updated)', 'GN2-18197 saved(updated)', 'GN7-17927 saved(updated)', 'GN2-18315 saved(updated)', 'GN7-17532 saved(updated)', 'GN7-19258 saved(updated)', 'GN7-19504 saved(updated)', 'GN2-21552 saved(updated)', 'GN4-21690 saved(updated)', 'GN4-22547 saved(up